# Stock Market Neutral Strategy 0002

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2019-12-31'
ed = '2026-02-13'

In [3]:
sql_factor =  f"""
SELECT
    date,
    instrument,
    ps_ttm AS factor
FROM cn_stock_prefactors
WHERE sw2021_level1 IN ('110000', '210000', '220000', '230000', '240000', '270000', '280000', '310000', '320000', '330000', '340000', '350000', '360000', '370000', '410000', '420000', '430000', '440000', '450000', '460000', '470000', '480000', '490000', '510000', '630000', '650000', '710000', '720000', '730000', '740000', '760000', '770000')
AND instrument NOT LIKE '%BJ%'
QUALIFY c_group_pct_rank(sw2021_level2, ps_ttm) < 0.20 
AND c_group_pct_rank(sw2021_level2, total_operating_revenue_yoy_lf) > 0.67
AND net_profit_yoy_lf > 0
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2019-12-31,600057.SH,0.033850,1.0,0.1
1,2019-12-31,600704.SH,0.078181,2.0,0.1
2,2019-12-31,600710.SH,0.083967,3.0,0.1
3,2019-12-31,000411.SZ,0.128704,4.0,0.1
4,2019-12-31,000829.SZ,0.132768,5.0,0.1
...,...,...,...,...,...
735,2026-01-30,000736.SZ,0.209529,6.0,0.1
736,2026-01-30,601607.SH,0.228557,7.0,0.1
737,2026-01-30,600697.SH,0.316284,8.0,0.1
738,2026-01-30,001287.SZ,0.327141,9.0,0.1


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader1_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader1_Before_Trading(context, data):
    pass

def BigTrader1_Handle_Tick(context, tick):
    pass

def BigTrader1_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader1_Handle_Trade(context, trade):
    pass

def BigTrader1_Handle_Order(context, order):
    pass

def BigTrader1_After_Trading(context, data):
    pass

BigTrader1 = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader1_Initialize,
    before_trading_start = BigTrader1_Before_Trading,
    handle_tick          = BigTrader1_Handle_Tick,
    handle_data          = BigTrader1_Handle_Data,
    handle_trade         = BigTrader1_Handle_Trade,
    handle_order         = BigTrader1_Handle_Order,
    after_trading        = BigTrader1_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader1"""
) 

[2026-02-27 11:32:48] [info     ] bigtrader.v34 开始运行 ..
[2026-02-27 11:32:48] [info     ] 2019-12-31, 2026-01-30, , , instruments=97
[2026-02-27 11:32:48] [info     ] bigtrader module V2.2.0
[2026-02-27 11:32:48] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '600470.SH' at '2020-04-30 15:00:00'
order_target_percent: price is null for '002146.SZ' at '2023-05-31 15:00:00'
[2026-02-27 11:32:54] [info     ] backtest done, raw_perf_ds:dai.DataSource("_2e08901303d84ed8b167cb13250b1269")


[2026-02-27 11:32:56] [info     ] bigtrader.v34 运行完成 [7.969s].


In [ ]:
from bigmodule import M

def BigTrader2_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.data_date = dai.query("SELECT date FROM mldt_cn_stock_calendar_daily WHERE is_month_end_trade = 1").df()

def BigTrader2_Before_Trading(context, data):
    pass

def BigTrader2_Handle_Tick(context, tick):
    pass

def BigTrader2_Handle_Data(context, data):

    df_now = context.data_date[context.data_date["date"] == data.current_dt.strftime("%Y-%m-%d")]
    if len(df_now) == 0:
        return

    from bigtrader.constant import OrderType
    from bigtrader.constant import Direction
    
    index_future = "IF8888.CFE"
    position_short = context.get_position(index_future, Direction.SHORT)
    price = data.current(index_future, "open")
    
    if (position_short.current_qty != 0):
        context.buy_close(index_future, position_short.avail_qty, price, order_type=OrderType.MARKET)
    
    context.sell_open(index_future, 1, price, order_type=OrderType.MARKET)

def BigTrader2_Handle_Trade(context, trade):
    pass

def BigTrader2_Handle_Order(context, order):
    pass

def BigTrader2_After_Trading(context, data):
    pass

BigTrader2 = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader2_Initialize,
    before_trading_start = BigTrader2_Before_Trading,
    handle_tick          = BigTrader2_Handle_Tick,
    handle_data          = BigTrader2_Handle_Data,
    handle_trade         = BigTrader2_Handle_Trade,
    handle_order         = BigTrader2_Handle_Order,
    after_trading        = BigTrader2_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""期货""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader2"""
) 

In [ ]:
df1 = BigTrader1.raw_perf.read()
df2 = BigTrader2.raw_perf.read()

In [ ]:
df1_new = df1[['date','portfolio_value']].rename(columns={'portfolio_value': 'portfolio_value_stock'}, inplace=False).reset_index(drop=True)
df2_new = df2[['date','portfolio_value']].rename(columns={'portfolio_value': 'portfolio_value_index'}, inplace=False).reset_index(drop=True)

In [ ]:
df_merge = pd.merge(df1_new, df2_new, on='date', how='inner')
df_merge['portfolio_value']       = df_merge['portfolio_value_stock'] + df_merge['portfolio_value_index']
df_merge['portfolio_value']       = df_merge['portfolio_value']       / df_merge['portfolio_value'].iloc[0]
df_merge['portfolio_value_stock'] = df_merge['portfolio_value_stock'] / df_merge['portfolio_value_stock'].iloc[0]
df_merge['portfolio_value_index'] = df_merge['portfolio_value_index'] / df_merge['portfolio_value_index'].iloc[0]

df_merge

,date,portfolio_value_stock,portfolio_value_index,portfolio_value
0,2019-12-31,1.000000,1.000000,1.000000
1,2020-01-02,0.999701,0.991212,0.995456
2,2020-01-03,1.000354,0.993792,0.997073
3,2020-01-06,0.994588,0.999551,0.997070
4,2020-01-07,1.000669,0.990612,0.995640
...,...,...,...,...
1471,2026-01-26,1.703665,0.819347,1.261505
1472,2026-01-27,1.683347,0.819707,1.251526
1473,2026-01-28,1.690937,0.815327,1.253131
1474,2026-01-29,1.710313,0.799967,1.255139


In [ ]:
from bigcharts import Line
Line(df_merge)